In [1]:
import sys

if sys.version_info[0] < 3:
    raise Exception('Python 3 or a more recent version is required.')

#from primitive_pb2 import can_set_my_account_detail
from irohalib import Iroha, IrohaGrpc
from irohalib import IrohaCrypto
import binascii

In [8]:
ADDRESS = 'localhost'
admin_private_key = open('admin@test.priv').read()
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key)
iroha = Iroha('admin@test')

In [3]:
def send_transaction_and_print_status(transaction, port):
    hex_hash = binascii.hexlify(IrohaCrypto.hash(transaction))
    print('Transaction hash = {}, creator = {}'.format(
        hex_hash, transaction.payload.reduced_payload.creator_account_id))
    net = IrohaGrpc(ADDRESS + ':' + str(port))
    net.send_tx(transaction)
    for status in net.tx_status_stream(transaction):
        print(status)

In [4]:
def create_domain_and_asset():
    """
    Creates domain 'domain' and asset 'coin#domain' with precision 2
    """
    commands = [
        iroha.command('CreateDomain', domain_id='domain', default_role='user'),
        iroha.command('CreateAsset', asset_name='coin',
                      domain_id='domain', precision=2)
    ]
    tx = IrohaCrypto.sign_transaction(
        iroha.transaction(commands), admin_private_key)
    send_transaction_and_print_status(tx, 50051)


In [5]:
def add_coin_to_admin():
    """
    Add 1000.00 units of 'coin#domain' to 'admin@test'
    """
    tx = iroha.transaction([
        iroha.command('AddAssetQuantity',
                      asset_id='coin#domain', amount='1000.00')
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx, 50051)


In [9]:
create_domain_and_asset()

Transaction hash = b'7e65ddf7993ca0cc36b5de3a09d9c39f9e2d18c96a8da766e220b01491fe69dc', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_FAILED', 2, 2)
('REJECTED', 4, 0)


In [ ]:
def get_account_assets():
    """
    List all the assets of userone@domain
    """
    query = iroha.query('GetAccountAssets', account_id='admin@test')
    IrohaCrypto.sign_query(query, admin_private_key)
    net = IrohaGrpc(ADDRESS + ':' + str(50051))
    response = net.send_query(query)
    data = response.account_assets_response.account_assets
    for asset in data:
        print('Asset id = {}, balance = {}'.format(
            asset.asset_id, asset.balance))

In [ ]:
get_account_assets()

In [10]:
user_private_key

b'2c0c4184858327f2fda2aa52580db19d23e6501b02b8cab79a6df53c04f345a8'

In [11]:
user_public_key

b'b477046973ae04be217983cd16acfdc669b007fd88dda341f0a7d43f9aa7e5bd'

In [37]:
#!/usr/bin/env python3
#
# Copyright Soramitsu Co., Ltd. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
#

import sys

if sys.version_info[0] < 3:
    raise Exception('Python 3 or a more recent version is required.')

from iroha.primitive_pb2 import can_set_my_account_detail, can_transfer_my_assets, can_transfer
from iroha import Iroha, IrohaGrpc
from iroha import IrohaCrypto
import binascii

admin_private_key = 'f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70'
iroha = Iroha('admin@test')
net = IrohaGrpc('127.0.0.1:50051')

In [2]:
def new_user(user_id):
    private_key = IrohaCrypto.private_key()
    if user_id.lower().startswith('admin'):
        print('K{}'.format(private_key.decode('utf-8')))
    return {
        'id': user_id,
        'key': private_key
    }

In [3]:
fabric = new_user('fabric@test')
office = new_user('office@test')

In [4]:
from iroha import primitive_pb2

In [ ]:
# def genesis_block(admin, alice, test_permissions, multidomain=False):
#     """
#     Compose a set of common for all tests' genesis block transactions
#     :param admin: dict of id and private key of admin
#     :param alice: dict of id and private key of alice
#     :param test_permissions: permissions for users in test domain
#     :param multidomain: admin and alice accounts will be created in
#     different domains and the first domain users will have admin right
#     by default if True
#     :return: a list of irohalib.Iroha.command's
#     """
#     peer = primitive_pb2.Peer()
#     peer.address = '0.0.0.0:50051'
#     peer.peer_key = admin_private_key
#     commands = [
#         command('AddPeer', peer=peer),
#         command('CreateRole', role_name='admin_role', permissions=all_permissions()),
#         command('CreateRole', role_name='test_role', permissions=test_permissions)]
#     if multidomain:
#         commands.append(command('CreateDomain', domain_id='first', default_role='admin_role'))
#     commands.extend([
#         command('CreateDomain',
#                 domain_id='second' if multidomain else 'test',
#                 default_role='test_role'),
#         command('CreateAccount',
#                 account_name='admin',
#                 domain_id='first' if multidomain else 'test',
#                 public_key=irohalib.IrohaCrypto.derive_public_key(admin['key'])),
#         command('CreateAccount',
#                 account_name='alice',
#                 domain_id='second' if multidomain else 'test',
#                 public_key=irohalib.IrohaCrypto.derive_public_key(alice['key']))
#     ])
#     if not multidomain:
#         commands.append(command('AppendRole', account_id=admin['id'], role_name='admin_role'))
#     return commands

In [27]:
test_permissions = [
        primitive_pb2.can_grant_can_transfer_my_assets,
        primitive_pb2.can_receive,
        primitive_pb2.can_transfer
    ]
genesis_commands = []#commons.genesis_block(admin, alice, test_permissions)
genesis_commands.extend([
#      iroha.command('CreateAccount', account_name='fabric', domain_id='test',
#                    public_key=IrohaCrypto.derive_public_key(fabric['key'])),
     #iroha.command('CreateAsset', asset_name='coin', domain_id='test', precision=2),
    #iroha.command('AddAssetQuantity', asset_id='coin#test', amount='100.00'),
    iroha.command('TransferAsset',
                  src_account_id='admin@test',
                  dest_account_id=fabric['id'],
                  asset_id='coin#test',
                  description='init top up',
                  amount='10.00')
])
tx = iroha.transaction(genesis_commands)
IrohaCrypto.sign_transaction(tx, admin_private_key)
send_transaction_and_print_status(tx)

	Entering "send_transaction_and_print_status"
Transaction hash = b'230e9a99d0a3ec2b2c33dfa29933a1d7a18f75f2134f17724afc0a6e6f2e3c00', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"


In [25]:
tx = iroha.transaction([
        iroha.command('GrantPermission', account_id=fabric['id'], permission=primitive_pb2.can_transfer_my_assets)
    ], creator_account='admin@test')
IrohaCrypto.sign_transaction(tx, admin_private_key)
send_transaction_and_print_status(tx)

	Entering "send_transaction_and_print_status"
Transaction hash = b'898372e0b5b80b0225ad9d0b9e0c7ec21a416a8b5231afb4a6f51b2acebd92a8', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"


In [42]:
iroha = Iroha('fabric@test')

In [33]:
tx = iroha.transaction([
        iroha.command('TransferAsset',
                      src_account_id=office['id'],
                      dest_account_id=fabric['id'],
                      asset_id='coin#test',
                      description='transfer from Alice to Admin by Bob',
                      amount='1.00')
    ])
IrohaCrypto.sign_transaction(tx, office['key'])
send_transaction_and_print_status(tx)

	Entering "send_transaction_and_print_status"
Transaction hash = b'a2a93c05613d3f15e821964b3e15c89dc9b519a686a68964a50ba566fbe986f8', creator = office@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"


In [6]:
def trace(func):
    """
    A decorator for tracing methods' begin/end execution points
    """

    def tracer(*args, **kwargs):
        name = func.__name__
        print('\tEntering "{}"'.format(name))
        result = func(*args, **kwargs)
        print('\tLeaving "{}"'.format(name))
        return result

    return tracer

In [7]:
@trace
def send_transaction_and_print_status(transaction):
    hex_hash = binascii.hexlify(IrohaCrypto.hash(transaction))
    print('Transaction hash = {}, creator = {}'.format(
        hex_hash, transaction.payload.reduced_payload.creator_account_id))
    net.send_tx(transaction)
    for status in net.tx_status_stream(transaction):
        print(status)

In [ ]:
@trace
def create_domain_and_asset():
    """
    Creates domain 'domain' and asset 'coin#domain' with precision 2
    """
    commands = [
        iroha.command('CreateDomain', domain_id='domain', default_role='user'),
        iroha.command('CreateAsset', asset_name='coin',
                      domain_id='domain', precision=2)
    ]
    tx = IrohaCrypto.sign_transaction(
        iroha.transaction(commands), admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def add_coin_to_admin():
    """
    Add 1000.00 units of 'coin#domain' to 'admin@test'
    """
    tx = iroha.transaction([
        iroha.command('AddAssetQuantity',
                      asset_id='coin#domain', amount='1000.00')
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def create_account_userone():
    """
    Create account 'userone@domain'
    """
    tx = iroha.transaction([
        iroha.command('CreateAccount', account_name='office', domain_id='domain',
                      public_key=user_public_key)
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def transfer_coin_from_admin_to_userone():
    """
    Transfer 2.00 'coin#domain' from 'admin@test' to 'userone@domain'
    """
    tx = iroha.transaction([
        iroha.command('TransferAsset', src_account_id='admin@test', dest_account_id='fabric@domain',
                      asset_id='coin#domain', description='init top up', amount='320.00')
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def userone_grants_to_admin_set_account_detail_permission():
    """
    Make admin@test able to set detail to userone@domain
    """
    tx = iroha.transaction([
        iroha.command('GrantPermission', account_id='admin@test',
                      permission=can_set_my_account_detail)
    ], creator_account='userone@domain')
    IrohaCrypto.sign_transaction(tx, user_private_key)
    send_transaction_and_print_status(tx)


@trace
def set_age_to_userone():
    """
    Set age to userone@domain by admin@test
    """
    tx = iroha.transaction([
        iroha.command('SetAccountDetail',
                      account_id='userone@domain', key='age', value='18')
    ])
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)


@trace
def get_coin_info():
    """
    Get asset info for coin#domain
    :return:
    """
    query = iroha.query('GetAssetInfo', asset_id='coin#domain')
    IrohaCrypto.sign_query(query, admin_private_key)

    response = net.send_query(query)
    data = response.asset_response.asset
    print('Asset id = {}, precision = {}'.format(data.asset_id, data.precision))

In [39]:
@trace
def get_account_assets():
    """
    List all the assets of userone@domain
    """
    query = iroha.query('GetAccountAssets', account_id='fabric@test', creator_account='fabric@test')
    IrohaCrypto.sign_query(query, office['key'])

    response = net.send_query(query)
    data = response.account_assets_response.account_assets
    for asset in data:
        print('Asset id = {}, balance = {}'.format(
            asset.asset_id, asset.balance))

In [85]:
@trace
def get_userone_details():
    """
    Get all the kv-storage entries for userone@domain
    """
    query = iroha.query('GetAccountDetail', account_id='userone@domain')
    IrohaCrypto.sign_query(query, admin_private_key)

    response = net.send_query(query)
    data = response.account_detail_response
    print('Account id = {}, details = {}'.format('userone@domain', data.detail))

In [77]:
create_domain_and_asset()

	Entering "create_domain_and_asset"
	Entering "send_transaction_and_print_status"
Transaction hash = b'286c4a8a39b25fbd34cf3efcadd3964ca9dbedbe1002e47e8254e7457467b9d2', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "create_domain_and_asset"


In [78]:
add_coin_to_admin()

	Entering "add_coin_to_admin"
	Entering "send_transaction_and_print_status"
Transaction hash = b'6a053b634829253b78f05ba7621ae9613d40a5e6b24fcb88147992ac0ae9a27c', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "add_coin_to_admin"


In [84]:
create_account_userone()

	Entering "create_account_userone"
	Entering "send_transaction_and_print_status"
Transaction hash = b'14f9dfa3f1ed99be7d57b861630a5e63c79863e5443bb5666603dd2f21526a5a', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "create_account_userone"


In [86]:
transfer_coin_from_admin_to_userone()

	Entering "transfer_coin_from_admin_to_userone"
	Entering "send_transaction_and_print_status"
Transaction hash = b'5fe94024b69e9e06c97a3bbadb7c48f53b9124750d65feb67182f74c34e297a2', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "transfer_coin_from_admin_to_userone"


In [15]:
userone_grants_to_admin_set_account_detail_permission()

	Entering "userone_grants_to_admin_set_account_detail_permission"
	Entering "send_transaction_and_print_status"
Transaction hash = b'1411d3d83e736e692a4e05c35d589809d82c34c01763fed22306421f7ff014fa', creator = userone@domain
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"
	Leaving "userone_grants_to_admin_set_account_detail_permission"


In [94]:
def userone_grants_to_admin_set_account_detail_permission():
    """
    Make admin@test able to set detail to userone@domain
    """
    tx = iroha.transaction([
        iroha.command('GrantPermission', account_id='fabric@domain',
                      permission=can_transfer_my_assets)
    ], creator_account='admin@test')
    IrohaCrypto.sign_transaction(tx, admin_private_key)
    send_transaction_and_print_status(tx)

In [95]:
userone_grants_to_admin_set_account_detail_permission()

	Entering "send_transaction_and_print_status"
Transaction hash = b'98790ea9beacc98a77d73c7755436cc9506837900004e0725682c24bffa91ea4', creator = admin@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "send_transaction_and_print_status"


In [ ]:
set_age_to_userone()
get_coin_info()

In [41]:
get_account_assets()

	Entering "get_account_assets"
	Leaving "get_account_assets"


In [45]:
def transfer_coin_from_admin_to_userone():
    """
    Transfer 2.00 'coin#domain' from 'admin@test' to 'userone@domain'
    """
    tx = iroha.transaction([
        iroha.command('TransferAsset', src_account_id='fabric@test', dest_account_id='office@test',
                      asset_id='good#test', description='init top up', amount='2.00')
    ])
    IrohaCrypto.sign_transaction(tx, "b79bd03b5e7521b9fe1fb42e5b10364d41db73dae1655cb67f892f254cc03b3b")
    send_transaction_and_print_status(tx)

In [46]:
transfer_coin_from_admin_to_userone()

	Entering "send_transaction_and_print_status"
Transaction hash = b'bd954d8e559bc67ef6b32fbea4a2fab859d0c956233ffdb769b211ed53c01048', creator = fabric@test
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_FAILED', 2, 2)
('REJECTED', 4, 0)
	Leaving "send_transaction_and_print_status"
